In [4]:
from odoo_transfer_lib import *

# Import Client and Session classes
from odoo_rpc_client import Client

In [12]:
cl_from = Client(host='localhost', port=11069, dbname='odoo11-odoo-test', user='admin', pwd='admin')
cl_to = Client(host='localhost', port=11169, dbname='test-data-transfer', user='admin', pwd='admin')

In [13]:
# Ensure connected
assert cl_from.uid
assert cl_to.uid

# Define models to transfer

In [16]:
class TResCurrency(TransferModel):
    model = 'res.currency'
    transfer_fields = ['name', 'symbol', 'active']
    link_field = 'name'
    auto_populate_cache = True
    

class TPricelistModel(TransferModel):
    model = 'product.pricelist'
    transfer_fields = ['active', 'name', 'currency_id']
    auto_populate_cache = True

    def get_search_domain(self, pricelist):
        return [('name', '=', pricelist.name),
                ('currency_id', '=', pricelist.currency_id.name)]


class TPartnerCategoryModel(TransferModel):
    model = 'res.partner.category'
    transfer_fields = [
        'active', 'name', 'parent_id'
    ]
    auto_populate_cache = True

    def get_search_domain(self, category):
        domain = [('name', '=', category.name)]
        level = 1
        parent = category.parent_id
        while parent:
            parent_field = 'parent_id.' * level + 'name'
            domain += [(parent_field, '=', parent.name)]
            parent = parent.parent_id
            level += 1
        return domain
    
class TCRMTeamModel(TransferModel):
    model = 'crm.team'

    transfer_fields = [
        'active', 'name', 'user_id',
    ]
    auto_populate_cache = True

    # link records by field 'code'
    link_field = 'name'


class TResPartnerModel(TransferModel):
    model = 'res.partner'
    auto_populate_cache = True
    populate_cache_domain = [('ref', '!=', False)]

    # auto_transfer_fields
    auto_transfer_enabled = False
    auto_transfer_domain = []
    auto_transfer_priority = 7
    # ---

    link_field = 'ref'

    transfer_fields = [
        'category_id', 'city', 'color', 'comment', 'country_id',
        'customer', 'date', 'email', 'function', 'is_company', 'mobile',
        'name', 'parent_id', 'phone', 'ref', 'street', 'street2', 'supplier', 'title',
        'type', 'tz', 'user_id', 'vat', 'website', 'zip',
        'active',
        # --- Pricelists
        'property_product_pricelist',

    ]

    def prepare_to_transfer(self, partner):
        super(TResPartnerModel, self).prepare_to_transfer(partner)
        if not partner.ref:
            x_ref = 'x-import-ref-%s' % partner.id
            partner.write({'ref': x_ref})
            partner._data['ref'] = x_ref  # little hack to not refresh each time


class TResCountryModel(TransferModel):
    model = 'res.country'
    auto_populate_cache = True
    link_field = 'code'


class TProductCategory(TransferModel):
    model = 'product.category'
    transfer_fields = [
        'name',
        'parent_id',
    ]
    auto_populate_cache = True

    def get_search_domain(self, category):
        domain = [('name', '=', category.name)]
        level = 1
        parent = category.parent_id
        while parent:
            parent_field = 'parent_id.' * level + 'name'
            domain += [(parent_field, '=', parent.name)]
            parent = parent.parent_id
            level += 1
        return domain
    
class TProductUOMModel(TransferModel):
    model = 'product.uom'
    auto_populate_cache = True
    link_field = 'name'

    
class TProductSupplierInfoModel(TransferModel):
    model = 'product.supplierinfo'
    transfer_fields = [
        'delay',
        'min_qty',
        'sequence',
        'name',
        'product_code',
        'product_name'
    ]

    
class TProductModel(TransferModel):
    model = 'product.product'
    transfer_fields = [
        'uom_po_id',
        'uom_id',
        'active',
        'purchase_ok',
        'categ_id',
        'description',
        'name',
        'description_sale',
        #'seller_ids',
        'volume',
        'type',
        'sale_ok',
        'default_code',
        'weight',
        'image',
        # ---
        # 'manufacturer', 'standard_price', 'list_price',
    ]
    link_field = 'default_code'

    auto_populate_cache = True
    populate_cache_domain = [('default_code', '!=', False)]

    # auto_transfer_fields
    auto_transfer_enabled = True
    auto_transfer_domain = []
    auto_transfer_priority = 10
    auto_transfer_xmlids = True
    # ---

    def prepare_to_transfer(self, product):
        super(TProductModel, self).prepare_to_transfer(product)
        if not product.default_code:
            default_code = "auto-code-%s" % product.id
            product.write({'default_code': default_code})
            product._data['default_code'] = default_code

            


# Transfer Data

In [17]:
transfer = Transfer(cl_from, cl_to, simplified_checks=True)
transfer.auto_transfer()
transfer.stat

IntProgress(value=0, description='             res.currency: cached [   0 of  170 records] (total 0.00 sec (0.…

IntProgress(value=0, description='        product.pricelist: cached [   0 of    1 records] (total 0.00 sec (0.…

IntProgress(value=0, description='     res.partner.category: cached [   0 of    0 records] (total 0.00 sec (0.…

IntProgress(value=0, description='                 crm.team: cached [   0 of    2 records] (total 0.00 sec (0.…

IntProgress(value=0, description='              res.partner: cached [   0 of    0 records] (total 0.00 sec (0.…

...

IntProgress(value=0, description='         product.category: cached [   0 of    2 records] (total 0.00 sec (0.…

IntProgress(value=0, description='              product.uom: cached [   0 of   19 records] (total 0.00 sec (0.…

IntProgress(value=0, description='          product.product: cached [   0 of    0 records] (total 0.00 sec (0.…

IntProgress(value=0, description='          product.product: transfered [   0 of   47 records] (total 0.00 sec…

{'created': {'res.currency': 0,
  'product.pricelist': 0,
  'res.partner.category': 0,
  'crm.team': 0,
  'res.partner': 0,
  'res.country': 0,
  'product.category': 3,
  'product.uom': 0,
  'product.supplierinfo': 0,
  'product.product': 47}}